# Downloading Data with VSO

In this session we will be using the Virtual Solar Observatory (VSO) cilent in SunPy to download some EUV data which we will use in the next session. The VSO has access to a wide variety of different instruments and data sources, and the SunPy client allows you to create arbitarily complex queries to search and then download the data.

## Useful Links

1. [Acquiring Data with SunPy](http://docs.sunpy.org/en/stable/guide/acquiring_data/index.html)
1. [The VSO](http://virtualsolar.org/)
1. [MapCube Documentation](http://docs.sunpy.org/en/stable/guide/data_types/maps.html#mapcubes)

## The VSO Module

SunPy's VSO submodule comes in two important parts, the `VSOClient` iteself, the thing that actually does the searching and donwloading of data and the 'attributes' which are things that allow you to build searches, they are things like `Time` or `Instrument`. We import them both: 

In [1]:
from sunpy.net import Fido, attrs as a
import astropy.units as u

Let's do a simple search for a day's worth of AIA data (don't worry, it wont download it all):

In [2]:
results = Fido.search(a.Time("2016/02/03", "2016/02/04"), a.Instrument('AIA'))

In [3]:
results

Start Time [1],End Time [1],Source,Instrument,Type,Wavelength [2]
,,,,,Angstrom
str19,str19,str3,str3,str8,float64
2016-02-03 19:49:22,2016-02-03 19:59:11,SDO,AIA,FULLDISK,211.0 .. 211.0
2016-02-03 19:59:29,2016-02-03 20:09:18,SDO,AIA,FULLDISK,193.0 .. 193.0
2016-02-03 18:09:24,2016-02-03 18:19:13,SDO,AIA,FULLDISK,94.0 .. 94.0
2016-02-03 04:09:34,2016-02-03 04:19:23,SDO,AIA,FULLDISK,171.0 .. 171.0
2016-02-03 14:39:30,2016-02-03 14:49:19,SDO,AIA,FULLDISK,304.0 .. 304.0
2016-02-03 11:49:31,2016-02-03 11:59:20,SDO,AIA,FULLDISK,131.0 .. 131.0
2016-02-03 07:39:31,2016-02-03 07:49:20,SDO,AIA,FULLDISK,131.0 .. 131.0
2016-02-03 19:29:31,2016-02-03 19:39:20,SDO,AIA,FULLDISK,131.0 .. 131.0


This is a little too much data, so let's restrict it to a single wavelength and ask for a lower cadence:

In [4]:
results = Fido.search(a.Time("2016/03/02", "2016/03/03"), a.Instrument('AIA'),
                      a.Wavelength(17.1*u.nm, 17.1*u.nm), a.Sample(2*u.h))

In [5]:
results

Start Time [1],End Time [1],Source,Instrument,Type,Wavelength [2]
,,,,,Angstrom
str19,str19,str3,str3,str8,float64
2016-03-02 22:00:10,2016-03-02 22:00:11,SDO,AIA,FULLDISK,171.0 .. 171.0
2016-03-02 06:00:10,2016-03-02 06:00:11,SDO,AIA,FULLDISK,171.0 .. 171.0
2016-03-02 16:00:10,2016-03-02 16:00:11,SDO,AIA,FULLDISK,171.0 .. 171.0
2016-03-02 08:00:10,2016-03-02 08:00:11,SDO,AIA,FULLDISK,171.0 .. 171.0
2016-03-02 02:00:10,2016-03-02 02:00:11,SDO,AIA,FULLDISK,171.0 .. 171.0
2016-03-02 18:00:10,2016-03-02 18:00:11,SDO,AIA,FULLDISK,171.0 .. 171.0
2016-03-02 20:00:10,2016-03-02 20:00:11,SDO,AIA,FULLDISK,171.0 .. 171.0
2016-03-02 10:00:10,2016-03-02 10:00:11,SDO,AIA,FULLDISK,171.0 .. 171.0


In [ ]:
files = Fido.fetch(results)

## SunPy Mapcube

A quick aside into plotting sequences of images with SunPy. We will load these files we just downloaded into a SunPy MapCube and animate them. (Note is a little more responsive outside of the notebook.)

In [7]:
%matplotlib notebook
import sunpy.map

In [8]:
mapcube = sunpy.map.Map(files, cube=True)

TypeError: buffer is too small for requested array

In [18]:
mapcube.peek()

NameError: name 'mapcube' is not defined

## More Complex Searching

In this example we will use the logical operators to combine attributes into a complex multi-instrument query.

You can use the `&` and operator or the `|` or operator to make queries. In the previous example all arguments to `vc.query` had the and operator applied, we performed a search where the Instrument *and* the Wavelength *and* the time matched. We could have done this explicitly like this:

In [19]:
mysearch = a.Time("2016/02/03", "2016/02/03T00:10:00") & a.Instrument('AIA')
Fido.search(mysearch)

Start Time [1],End Time [1],Source,Instrument,Type,Wavelength [2]
,,,,,Angstrom
str19,str19,str3,str3,str8,float64
2016-02-03 00:00:00,2016-02-03 00:00:01,SDO,AIA,FULLDISK,94.0 .. 94.0
2016-02-03 00:00:01,2016-02-03 00:00:02,SDO,AIA,FULLDISK,335.0 .. 335.0
2016-02-03 00:00:05,2016-02-03 00:00:06,SDO,AIA,FULLDISK,193.0 .. 193.0
2016-02-03 00:00:06,2016-02-03 00:00:07,SDO,AIA,FULLDISK,304.0 .. 304.0
2016-02-03 00:00:06,2016-02-03 00:00:07,SDO,AIA,FULLDISK,4500.0 .. 4500.0
2016-02-03 00:00:07,2016-02-03 00:00:08,SDO,AIA,FULLDISK,131.0 .. 131.0
2016-02-03 00:00:10,2016-02-03 00:00:11,SDO,AIA,FULLDISK,171.0 .. 171.0
2016-02-03 00:00:10,2016-02-03 00:00:11,SDO,AIA,FULLDISK,211.0 .. 211.0


If we want to do the same query but for two seperate wavelengths we can use the `|` or operator:

In [20]:
instrument = a.Time("2016/02/03", "2016/02/03T00:10:00") & a.Instrument('AIA')
wavelength = a.Wavelength(17.1*u.nm, 17.1*u.nm) | a.Wavelength(304*u.AA, 304*u.AA)
mysearch = instrument & wavelength

In [21]:
Fido.search(mysearch)

Start Time [1],End Time [1],Source,Instrument,Type,Wavelength [2]
,,,,,Angstrom
str19,str19,str3,str3,str8,float64
2016-02-03 00:00:10,2016-02-03 00:00:11,SDO,AIA,FULLDISK,171.0 .. 171.0
2016-02-03 00:00:22,2016-02-03 00:00:23,SDO,AIA,FULLDISK,171.0 .. 171.0
2016-02-03 00:00:34,2016-02-03 00:00:35,SDO,AIA,FULLDISK,171.0 .. 171.0
2016-02-03 00:00:46,2016-02-03 00:00:47,SDO,AIA,FULLDISK,171.0 .. 171.0
2016-02-03 00:00:58,2016-02-03 00:00:59,SDO,AIA,FULLDISK,171.0 .. 171.0
2016-02-03 00:01:10,2016-02-03 00:01:11,SDO,AIA,FULLDISK,171.0 .. 171.0
2016-02-03 00:01:22,2016-02-03 00:01:23,SDO,AIA,FULLDISK,171.0 .. 171.0
2016-02-03 00:01:34,2016-02-03 00:01:35,SDO,AIA,FULLDISK,171.0 .. 171.0


### Multiple Instruments

In this example we want to download one image from STEREO A EUVI and one image from SDO AIA as close together in time as we can. To do this we will define a search for the AIA image and a search for the EUVI image then 'or' them together.

In [24]:
stereo = (a.Detector('STEREO_B') &
          a.Instrument('EUVI') &
          a.Time('2011-01-01', '2011-01-01T00:10:00'))

aia = (a.Instrument('AIA') &
       a.Sample(24 * u.hour) &
       a.Time('2011-01-01', '2011-01-02'))

wave = a.Wavelength(30 * u.nm, 31 * u.nm)

In [26]:
results = Fido.search(stereo | aia, wave)
results

<sunpy.net.fido_factory.UnifiedResponse object at 0x7f6bdd14b6d8>
Results from 2 Providers:

0 Results from the VSOClient:
Start Time End Time  Source Instrument   Type 
 float64   float64  float64  float64   float64
---------- -------- ------- ---------- -------

1 Results from the VSOClient:
   Start Time [1]       End Time [1]    Source ...   Type   Wavelength [2]
                                               ...             Angstrom   
       str19               str19         str3  ...   str8      float64    
------------------- ------------------- ------ ... -------- --------------
2011-01-01 00:00:08 2011-01-01 00:00:09    SDO ... FULLDISK 304.0 .. 304.0

In [ ]:
files = Fido.fetch(result).wait(progress=True)

In [ ]:
files